In [1]:
import time
import os

import scanpy
import numpy as np
import scipy.sparse as sp

import torch
from torch import optim
from torch.utils.data import DataLoader

import models.loadImg as loadImg
import models.modelsCNN as modelsCNN
import models.optimizer as optimizer

import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import gc
from skimage import io
import umap
from sklearn.cluster import MiniBatchKMeans,AgglomerativeClustering
from sklearn.decomposition import PCA

In [2]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "0,1,2,3" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3" 
use_cuda=True

In [3]:
radius=48
datadir='/media/xinyi/dcis2idc/data'
sampleList_new=[]
sampleList=[]
for s in os.listdir('/media/xinyi/dcis2idc/data'):
    if 'hoechst'  in s and ('_1_' in s or '_3_' in s or '_4_' in s or '_6_' in s):
        sampleList.append(s)
    elif 'hoechst' in s:
        sampleList_new.append(s)
print(sampleList)
print(sampleList_new)
trainingCores={'br1003a':[],'br8018a':[],'br301':[]}
for s in range(1,11):
# for s in range(5,11):
    trainingCores['br1003a'].append('B'+str(s))
    trainingCores['br1003a'].append('D'+str(s))
    trainingCores['br1003a'].append('E'+str(s))
    trainingCores['br1003a'].append('F'+str(s))
    trainingCores['br1003a'].append('J'+str(s))
trainingCores['br1003a'].append('J'+str(11))
for s in range(1,11):
    trainingCores['br8018a'].append('C'+str(s))
    trainingCores['br8018a'].append('D'+str(s))
    trainingCores['br8018a'].append('E'+str(s))
    trainingCores['br8018a'].append('G'+str(s))
trainingCores['br8018a'].append('H'+str(9))
trainingCores['br8018a'].append('H'+str(10))


trainingCores_new={'br1003a':[],'br8018a':[],'br301':[]}
for s in range(1,7):
    for sr in ['A','B','C','D','E']:
        trainingCores_new['br301'].append(sr+str(s))
for s in range(1,11):
    for sr in ['A','B','C','D','E','F','I','J']:
        trainingCores_new['br1003a'].append(sr+str(s))
for s in range(1,11):
    for sr in ['A','B','C','D','E','F','G','H']:
        trainingCores_new['br8018a'].append(sr+str(s))

['br1003a_1_cytokeratin_555_aSMA_647_hoechst', 'br1003a_3_collagen1_647_hoechst', 'br1003a_4_cytokeratin_555_gh2ax_647_hoechst', 'br301_4_cytokeratin_555_aSMA_647_hoechst', 'br301_6_collagen1_647_hoechst', 'br8018a_1_cytokeratin_555_aSMA_647_hoechst', 'br8018a_3_collagen1_647_hoechst', 'br8018a_4_cytokeratin_555_gh2ax_647_hoechst']
['br1003a_2_cytokeratin_555_ki67_647_hoechst', 'br301_5_cytokeratin_555_ki67_647_hoechst', 'br301_7_gh2ax_647_cytokeratin_555_hoechst', 'br8018a_2_cytokeratin_555_ki67_647_hoechst']


In [5]:
datadir='/media/xinyi/dcis2idc/data'
allImg=loadImg.loadImg(datadir,sampleList,trainingCores,'segmented_nucleus','zproject/dna',radius,minmax=True)
with open(os.path.join(datadir,'processed','train_cnnvae_valcores'), 'wb') as output:
    pickle.dump(allImg, output, pickle.HIGHEST_PROTOCOL)

br1003a_1_cytokeratin_555_aSMA_647_hoechst
B1
D1
J1
B2
D2
J2
B3
D3
J3
B4
D4
J4
B5
J6
B7
D7
J7
B8
J8
B9
J9
B10
J10
br1003a_3_collagen1_647_hoechst
B1
D1
J1
B2
D2
J2
B3
D3
J3
D4
J4
B5
D5
J5
D6
J6
B7
D7
J7
B8
D8
J8
B9
D9
J9
B10
D10
J10
J11
br1003a_4_cytokeratin_555_gh2ax_647_hoechst
B1
D1
B2
D2
B3
D3
B4
D4
B5
D5
B6
D6
B7
D7
B8
D8
B9
D9
B10
D10
br301_4_cytokeratin_555_aSMA_647_hoechst
br301_6_collagen1_647_hoechst
br8018a_1_cytokeratin_555_aSMA_647_hoechst
C1
D1
E1
G1
C2
D2
E2
G2
C3
D3
E3
G3
C4
D4
E4
G4
C5
D5
E5
G5
C6
D6
E6
G6
C7
D7
E7
G7
C8
D8
E8
G8
C9
D9
E9
G9
C10
D10
E10
G10
H9
br8018a_3_collagen1_647_hoechst
C1
D1
E1
G1
C2
D2
E2
G2
D3
E3
G3
C4
D4
G4
C5
D5
E5
G5
C6
D6
G6
C7
D7
C8
D8
E8
G8
C9
D9
E9
C10
D10
E10
G10
br8018a_4_cytokeratin_555_gh2ax_647_hoechst
C1
D1
E1
C2
D3
E3
C4
G4
C5
D5
G5
C6
E6
G6
C7
E7
G7
C8
D8
E8
G8
C9
D9
E9
H9
H10


In [6]:
allImg_new=loadImg.loadImg(datadir,sampleList_new,trainingCores_new,'segmented_nucleus','zproject/dna',radius,minmax=True)
with open(os.path.join(datadir,'processed','train_cnnvae_valsamples'), 'wb') as output:
    pickle.dump(allImg_new, output, pickle.HIGHEST_PROTOCOL)

br1003a_2_cytokeratin_555_ki67_647_hoechst
A1
B1
C1
D1
I1
J1
A2
B2
C2
D2
I2
J2
A3
B3
C3
D3
I3
J3
B4
C4
D4
I4
J4
A5
B5
C5
D5
I5
B6
C6
D6
J6
A7
B7
C7
D7
I7
A8
B8
C8
D8
I8
B9
C9
I9
A10
B10
C10
D10
I10
br301_5_cytokeratin_555_ki67_647_hoechst
A1
B1
C1
D1
E1
A2
B2
C2
D2
E2
A3
B3
C3
D3
E3
A4
B4
C4
D4
E4
A5
B5
C5
D5
E5
A6
B6
C6
D6
E6
br301_7_gh2ax_647_cytokeratin_555_hoechst
A1
B1
C1
D1
E1
A2
B2
C2
D2
E2
A3
B3
C3
D3
E3
A4
B4
C4
D4
E4
A5
B5
C5
D5
E5
A6
B6
C6
E6
br8018a_2_cytokeratin_555_ki67_647_hoechst
A1
B1
C1
D1
E1
F1
G1
A2
B2
C2
E2
F2
G2
H2
A3
B3
C3
D3
E3
F3
G3
H3
A4
C4
D4
E4
F4
G4
H4
A5
B5
C5
D5
E5
F5
G5
H5
A6
B6
C6
D6
E6
F6
G6
A7
B7
C7
D7
E7
F7
A8
B8
C8
E8
F8
A9
B9
C9
E9
F9
A10
B10
C10
D10
E10


In [7]:
#VAE settings
seed=3
epochs=10000
saveFreq=2000
lr=0.00001 #initial learning rate
lr_adv=0.001
weight_decay=0 #Weight for L2 loss on embedding matrix.

# batchsize=4
batchsize=8000
kernel_size=4
stride=2
padding=1

# fc_dim1=6000
hidden1=64 #Number of channels in hidden layer 1
hidden2=128 
hidden3=256
hidden4=256
hidden5=96
fc_dim1=96*3*3
fc_dim2=6000
# fc_dim3=128
# fc_dim4=128
# gcn_dim1=2600
# adv_hidden=128

dropout=0.01
kl_weight=0.0000001
model_str='cnn_vae'

In [8]:
name='exp0'
logsavepath='/media/xinyi/dcis2idc/log/cnnvae'+name
modelsavepath='/media/xinyi/dcis2idc/models/cnnvae'+name
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name

In [9]:
# Set cuda and seed
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.enabled = True
    
loss_match=torch.nn.MSELoss()
# Create model
if model_str=='cnn_vae':
    modelcnn = modelsCNN.CNN_VAE(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,fc_dim2)
    loss_kl=optimizer.optimizer_kl
    loss_x=torch.nn.MSELoss()
if model_str=='cnn_vae_sharded':
    modelcnn = modelsCNN.CNN_VAE_sharded(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,fc_dim2)
    loss_kl=optimizer.optimizer_kl
    loss_x=torch.nn.MSELoss()
    use_cuda=False

if model_str=='cnn_vae_alexnet':
    modelcnn = modelsCNN.CNN_VAE_alexnet(fc_dim1)
    loss_kl=optimizer.optimizer_kl
    loss_x=torch.nn.MSELoss(reduction='sum')

    
modelcnn.cuda()       
optimizerCNN = optim.Adam(modelcnn.parameters(), lr=lr, weight_decay=weight_decay)

ep=311

In [10]:
#compute latent

use_cuda=True
modelcnn.load_state_dict(torch.load(os.path.join(modelsavepath,str(ep)+'.pt')))
latent=np.zeros((allImg.shape[0],fc_dim2))
with torch.no_grad():
    modelcnn.cuda()
    modelcnn.eval()
    nplotBatches=int(np.ceil(allImg.shape[0]/batchsize))
    for i in range(nplotBatches):
        plotInput=torch.tensor(allImg[i*batchsize:min((i+1)*batchsize,allImg.shape[0])])
        if use_cuda:
            plotInput=plotInput.cuda().float()
        recon,z, mu, logvar = modelcnn(plotInput)
        latent[i*batchsize:min((i+1)*batchsize,allImg.shape[0])]=mu.cpu().detach().numpy()



In [11]:
#compute latent_new - new samples

latent_new=np.zeros((allImg_new.shape[0],fc_dim2))
with torch.no_grad():
    modelcnn.cuda()
    modelcnn.eval()
    nplotBatches=int(np.ceil(allImg_new.shape[0]/batchsize))
    for i in range(nplotBatches):
        plotInput=torch.tensor(allImg_new[i*batchsize:min((i+1)*batchsize,allImg_new.shape[0])])
        if use_cuda:
            plotInput=plotInput.cuda().float()
        recon,z, mu, logvar = modelcnn(plotInput)
        latent_new[i*batchsize:min((i+1)*batchsize,allImg_new.shape[0])]=mu.cpu().detach().numpy()


In [12]:
with open(os.path.join(datadir,'processed','latent311_valcores'), 'wb') as input:
    pickle.dump(latent,input,pickle.HIGHEST_PROTOCOL)
with open(os.path.join(datadir,'processed','latent311_valsamples'), 'wb') as input:
    pickle.dump(latent_new,input,pickle.HIGHEST_PROTOCOL)

In [13]:
#get sample names
def loadOneSampleName(samplename,segmentationPath,imgPath,centroidPath,radius,minmax):
    segmentednuclei=io.imread(segmentationPath)
    pos=pd.read_csv(centroidPath)
    rep=pos.shape[0]
    for i in range(pos.shape[0]):
        if int(pos.iloc[i,2])-radius<0 or int(pos.iloc[i,2])+radius>(segmentednuclei.shape[0]) or int(pos.iloc[i,3])-radius<0 or int(pos.iloc[i,3])+radius>(segmentednuclei.shape[1]):
            rep-=1
    sampleNameList=np.repeat(samplename,rep)
    return sampleNameList
    
def loadImgName(datadir,sampleList,coreList,segmentationPath,imgPath,radius,minmax=True):
    allImg=None
    
    for s in sampleList:
        print(s)
        coreS=None
        for k in coreList.keys():
            if k in s:
                coreS=coreList[k]
                break
        for c in coreS:
            if not os.path.exists(os.path.join(datadir,s,segmentationPath,c+'.tif')) or (not os.path.exists(os.path.join(datadir,s,imgPath,c+'.tif'))) or (not os.path.exists(os.path.join(datadir,s,'spatial_positioning',c+'.csv'))):
                continue
            print(c)
            samplename=s+'_'+c
            if allImg is None:
                allImg=loadOneSampleName(samplename,os.path.join(datadir,s,segmentationPath,c+'.tif'), os.path.join(datadir,s,imgPath,c+'.tif'), os.path.join(datadir,s,'spatial_positioning',c+'.csv'), radius,minmax)
            else:
                allImg = np.concatenate((allImg, loadOneSampleName(samplename,os.path.join(datadir,s,segmentationPath,c+'.tif'), os.path.join(datadir,s,imgPath,c+'.tif'), os.path.join(datadir,s,'spatial_positioning',c+'.csv'), radius,minmax)), axis=0)
    return allImg

gc.collect()
allImgNames=loadImgName(datadir,sampleList,trainingCores,'segmented_nucleus','zproject/dna',radius,minmax=True)
with open(os.path.join(datadir,'processed','train_cnnvae_names_valcores'), 'wb') as output:
    pickle.dump(allImgNames, output, pickle.HIGHEST_PROTOCOL)
allImgNames_new=loadImgName(datadir,sampleList_new,trainingCores_new,'segmented_nucleus','zproject/dna',radius,minmax=True)
with open(os.path.join(datadir,'processed','train_cnnvae_names_valsamples'), 'wb') as output:
    pickle.dump(allImgNames_new, output, pickle.HIGHEST_PROTOCOL)

gc.collect()

br1003a_1_cytokeratin_555_aSMA_647_hoechst
B1
D1
J1
B2
D2
J2
B3
D3
J3
B4
D4
J4
B5
J6
B7
D7
J7
B8
J8
B9
J9
B10
J10
br1003a_3_collagen1_647_hoechst
B1
D1
J1
B2
D2
J2
B3
D3
J3
D4
J4
B5
D5
J5
D6
J6
B7
D7
J7
B8
D8
J8
B9
D9
J9
B10
D10
J10
J11
br1003a_4_cytokeratin_555_gh2ax_647_hoechst
B1
D1
B2
D2
B3
D3
B4
D4
B5
D5
B6
D6
B7
D7
B8
D8
B9
D9
B10
D10
br301_4_cytokeratin_555_aSMA_647_hoechst
br301_6_collagen1_647_hoechst
br8018a_1_cytokeratin_555_aSMA_647_hoechst
C1
D1
E1
G1
C2
D2
E2
G2
C3
D3
E3
G3
C4
D4
E4
G4
C5
D5
E5
G5
C6
D6
E6
G6
C7
D7
E7
G7
C8
D8
E8
G8
C9
D9
E9
G9
C10
D10
E10
G10
H9
br8018a_3_collagen1_647_hoechst
C1
D1
E1
G1
C2
D2
E2
G2
D3
E3
G3
C4
D4
G4
C5
D5
E5
G5
C6
D6
G6
C7
D7
C8
D8
E8
G8
C9
D9
E9
C10
D10
E10
G10
br8018a_4_cytokeratin_555_gh2ax_647_hoechst
C1
D1
E1
C2
D3
E3
C4
G4
C5
D5
G5
C6
E6
G6
C7
E7
G7
C8
D8
E8
G8
C9
D9
E9
H9
H10
br1003a_2_cytokeratin_555_ki67_647_hoechst
A1
B1
C1
D1
I1
J1
A2
B2
C2
D2
I2
J2
A3
B3
C3
D3
I3
J3
B4
C4
D4
I4
J4
A5
B5
C5
D5
I5
B6
C6
D6
J6
A7
B7
C7
D7
I7
A

235

In [14]:
#get coord
coordlist=np.zeros((allImgNames.shape[0],2))
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)
for sidx in range(uniquenames.size):
    s=uniquenames[sidx]
    startidx=nameIdx[sidx]
    posPath=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'spatial_positioning',s.split('_')[-1] +'.csv')
    pos=pd.read_csv(posPath)
    
    segmentationPath=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'segmented_nucleus',s.split('_')[-1] +'.tif')
    segmentednuclei=io.imread(segmentationPath)
    for i in range(pos.shape[0]):
        if int(pos.iloc[i,2])-radius<0 or int(pos.iloc[i,2])+radius>(segmentednuclei.shape[0]) or int(pos.iloc[i,3])-radius<0 or int(pos.iloc[i,3])+radius>(segmentednuclei.shape[1]):
            continue
        coordlist[startidx]=np.array((pos.iloc[i,2],pos.iloc[i,3]))
        startidx+=1

with open(os.path.join(datadir,'processed','train_cnnvae_coord_valcores'), 'wb') as output:
    pickle.dump(coordlist, output, pickle.HIGHEST_PROTOCOL)

In [15]:
#get coord
coordlist_new=np.zeros((allImgNames_new.shape[0],2))
uniquenames,nameIdx=np.unique(allImgNames_new,return_index=True)
for sidx in range(uniquenames.size):
    s=uniquenames[sidx]
    startidx=nameIdx[sidx]
    posPath=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'spatial_positioning',s.split('_')[-1] +'.csv')
    pos=pd.read_csv(posPath)
    
    segmentationPath=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'segmented_nucleus',s.split('_')[-1] +'.tif')
    segmentednuclei=io.imread(segmentationPath)
    for i in range(pos.shape[0]):
        if int(pos.iloc[i,2])-radius<0 or int(pos.iloc[i,2])+radius>(segmentednuclei.shape[0]) or int(pos.iloc[i,3])-radius<0 or int(pos.iloc[i,3])+radius>(segmentednuclei.shape[1]):
            continue
        coordlist_new[startidx]=np.array((pos.iloc[i,2],pos.iloc[i,3]))
        startidx+=1

with open(os.path.join(datadir,'processed','train_cnnvae_coord_valsamples'), 'wb') as output:
    pickle.dump(coordlist_new, output, pickle.HIGHEST_PROTOCOL)

In [16]:
#get cellID -- label not row index
cellIDlist={}
uniquenames,nameIdx=np.unique(allImgNames,return_index=True)
for sidx in range(uniquenames.size):
    s=uniquenames[sidx]
    posPath=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'spatial_positioning',s.split('_')[-1] +'.csv')
    pos=pd.read_csv(posPath)
    
    includedPos=coordlist[allImgNames==s]
    cellIDlist[s]=np.zeros(includedPos.shape[0])
    for i in range(includedPos.shape[0]):
        cellIDlist[s][i]=pos.to_numpy()[np.logical_and(pos.iloc[:,2]==includedPos[i,0],pos.iloc[:,3]==includedPos[i,1]),1]
    assert np.sum(cellIDlist[s]==0)==0
    
with open(os.path.join(datadir,'processed','train_cnnvae_cellLabels_valcores'), 'wb') as output:
    pickle.dump(cellIDlist,output, pickle.HIGHEST_PROTOCOL)

In [18]:
#get cellID -- label not row index
cellIDlist_new={}
uniquenames,nameIdx=np.unique(allImgNames_new,return_index=True)
for sidx in range(uniquenames.size):
    s=uniquenames[sidx]
    posPath=os.path.join(datadir,'_'.join(s.split('_')[:-1]),'spatial_positioning',s.split('_')[-1] +'.csv')
    pos=pd.read_csv(posPath)
    
    includedPos=coordlist_new[allImgNames_new==s]
    cellIDlist_new[s]=np.zeros(includedPos.shape[0])
    for i in range(includedPos.shape[0]):
        cellIDlist_new[s][i]=pos.to_numpy()[np.logical_and(pos.iloc[:,2]==includedPos[i,0],pos.iloc[:,3]==includedPos[i,1]),1]
    assert np.sum(cellIDlist_new[s]==0)==0
    
with open(os.path.join(datadir,'processed','train_cnnvae_cellLabels_valsamples'), 'wb') as output:
    pickle.dump(cellIDlist_new,output, pickle.HIGHEST_PROTOCOL)

In [25]:
allImg_new=None
gc.collect()
allImg=None
gc.collect()

0

In [4]:
#load
ep=311
name='exp0'
logsavepath='/media/xinyi/dcis2idc/log/cnnvae'+name
modelsavepath='/media/xinyi/dcis2idc/models/cnnvae'+name
plotsavepath='/media/xinyi/dcis2idc/plots/cnnvae'+name

with open(os.path.join(datadir,'processed','latent311_valcores'), 'rb') as input:
    latent=pickle.load(input)
with open(os.path.join(datadir,'processed','latent311_valsamples'), 'rb') as input:
    latent_new=pickle.load(input)

In [7]:
clustersavedir_valcores=os.path.join(sampledir,'cluster_valcores_reordered')
clusterplotdir_valcores=os.path.join(clustersavedir_valcores,'plots')
clustersavedir_valsamples=os.path.join(sampledir,'cluster_valsamples_reordered')
clusterplotdir_valsamples=os.path.join(clustersavedir_valsamples,'plots')

plotepoch=ep
ncluster=8
n_neighbors=10
min_dist=0.25
n_pcs=50
savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)
with open(os.path.join(clustersavedir_valcores,savenamecluster+'_all'), 'rb') as output:
    clusterRes_reordered=pickle.load(output)
with open(os.path.join(clustersavedir_valsamples,'minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'rb') as output:
    clusterRes_reordered_new=pickle.load(output)

In [9]:
plottype='umap'
sampledir=plotsavepath
savedir=os.path.join(sampledir,'embedding_'+plottype)
clustersavedir=os.path.join(sampledir,'cluster')
neworder=[1, 5, 3, 7, 2, 0, 4, 6]

In [6]:
#load umap and kmeans -- save reordered cluster
ncluster=8
n_neighbors=10
min_dist=0.25
n_pcs=50
plotepoch=ep
# savenameAdd='_nn'+str(n_neighbors)+'mdist0'+str(int(min_dist*100))+'epoch'+str(ep)
# with open(os.path.join(modelsavepath,'umap_plottingIdx_progBalanced_'+str(0)+savenameAdd), 'rb') as output:
#     reducer=pickle.load(output)

savenameAdd='_plottingIdx_progBalanced_'+str(0)
savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+savenameAdd
with open(os.path.join(clustersavedir,savenamecluster+'_estimator'), 'rb') as output:
    kmeansestimator=pickle.load(output)

savenamepca='pca_epoch'+str(plotepoch)+savenameAdd
with open(os.path.join(clustersavedir,savenamepca), 'rb') as output:
    pca=pickle.load(output)
    
subclusternumbers=[4,6,8,6,6,6,6,4]
kmeansestimator_sub={}
pca_sub={}
for c in range(ncluster):
    subclustersavedir=os.path.join(clustersavedir,savenamecluster+'_subcluster'+str(c))
    savenamecluster_sub='minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+savenameAdd
    with open(os.path.join(subclustersavedir,savenamecluster_sub+'_estimator'), 'rb') as output:
        kmeansestimator_sub[c]=pickle.load(output)
    with open(os.path.join(subclustersavedir,savenamepca), 'rb') as output:
        pca_sub[c]=pickle.load(output)

In [33]:
#predict kmeans
clustersavedir_valcores=os.path.join(sampledir,'cluster_valcores_reordered')
if not os.path.exists(clustersavedir_valcores):
    os.mkdir(clustersavedir_valcores)
clusterplotdir_valcores=os.path.join(clustersavedir_valcores,'plots')
if not os.path.exists(clusterplotdir_valcores):
    os.mkdir(clusterplotdir_valcores)
clusterRes=kmeansestimator.predict(pca.transform(latent)[:,:n_pcs])
clusterRes_reordered=np.zeros_like(clusterRes)
for c in range(ncluster):
    cold=neworder[c]
    clusterRes_reordered[clusterRes==cold]=c

with open(os.path.join(clustersavedir_valcores,'minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'wb') as output:
    pickle.dump(clusterRes_reordered, output, pickle.HIGHEST_PROTOCOL)

gc.collect()


21439

In [15]:
savenamecluster='minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+savenameAdd
for cnew in np.unique(clusterRes_reordered):
    c=neworder[cnew]
    subclustersavedir_valcores=os.path.join(clustersavedir_valcores,savenamecluster+'_subcluster'+str(c))
    subclusterplotdir_valcores=os.path.join(subclustersavedir_valcores,'plots')
    if not os.path.exists(subclustersavedir_valcores):
        os.mkdir(subclustersavedir_valcores)
    if not os.path.exists(subclusterplotdir_valcores):
        os.mkdir(subclusterplotdir_valcores)
    clusterRes_sub=kmeansestimator_sub[c].predict(pca_sub[c].transform(latent[clusterRes_reordered==cnew])[:,:n_pcs])
    with open(os.path.join(subclustersavedir_valcores,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'wb') as output:
        pickle.dump(clusterRes_sub, output, pickle.HIGHEST_PROTOCOL)



In [35]:
#predict kmeans
clustersavedir_valsamples=os.path.join(sampledir,'cluster_valsamples_reordered')
if not os.path.exists(clustersavedir_valsamples):
    os.mkdir(clustersavedir_valsamples)
clusterplotdir_valsamples=os.path.join(clustersavedir_valsamples,'plots')
if not os.path.exists(clusterplotdir_valsamples):
    os.mkdir(clusterplotdir_valsamples)
clusterRes_new=kmeansestimator.predict(pca.transform(latent_new)[:,:n_pcs])
clusterRes_reordered_new=np.zeros_like(clusterRes_new)
for c in range(ncluster):
    cold=neworder[c]
    clusterRes_reordered_new[clusterRes_new==cold]=c

with open(os.path.join(clustersavedir_valsamples,'minibatchkmean_ncluster'+str(ncluster)+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'wb') as output:
    pickle.dump(clusterRes_reordered_new, output, pickle.HIGHEST_PROTOCOL)

gc.collect()
# embedding=reducer.transform(latent)

0

In [17]:
for cnew in np.unique(clusterRes_reordered_new):
    c=neworder[cnew]
    subclustersavedir_valsamples=os.path.join(clustersavedir_valsamples,savenamecluster+'_subcluster'+str(c))
    subclusterplotdir_valsamples=os.path.join(subclustersavedir_valsamples,'plots')
    if not os.path.exists(subclustersavedir_valsamples):
        os.mkdir(subclustersavedir_valsamples)
    if not os.path.exists(subclusterplotdir_valsamples):
        os.mkdir(subclusterplotdir_valsamples)
    clusterRes_sub_new=kmeansestimator_sub[c].predict(pca_sub[c].transform(latent_new[clusterRes_reordered_new==cnew])[:,:n_pcs])
    with open(os.path.join(subclustersavedir_valsamples,'minibatchkmean_ncluster'+str(subclusternumbers[c])+'n_pcs'+str(n_pcs)+'epoch'+str(plotepoch)+'_all'), 'wb') as output:
        pickle.dump(clusterRes_sub_new, output, pickle.HIGHEST_PROTOCOL)


